In [1]:
import numpy as np
import time as Time
from numba import jit
import matplotlib.pyplot as plt
import pyvista as pv
import torch

In [27]:
#solid = film[i, j, k, 10][Si, SiF1, SiF2, SiF3, SiO SiO2, SiOF, SiOF2, SiO2F, SiO2F2]
#react_t g[F, O, ion] s  [1,          2,           3,          4,       5 ,   6,    7,    8,   9,  10]
#react_t g[F, O, ion] s  [Si,       SiF1,       SiF2,       SiF3,      SiO, SiO2, SiOF, SiOF2, SiO2F,SiO2F2]


film = np.zeros((50, 50, 100, 10))

bottom = 60
film[:, :, 0:bottom, 0] = 10 # bottom

height = 85

film[:, :15, bottom:height, 5] = 10
film[:, 35:, bottom:height, 5] = 10

slit = np.linspace(25, 0, 5)
for k in range(5):
    film[:, 15+k, bottom:int(slit[k]) + bottom, 5] = 10
    film[:, 30+k, bottom:int(slit[4-k]) + bottom, 5] = 10


In [15]:
film = np.zeros((50, 50, 100, 10))

bottom = 80
film[:, :, 0:bottom, 0] = 10 # bottom

height = 85

film[:, :22, bottom:height, 5] = 10
film[:, 28:, bottom:height, 5] = 10

In [3]:
# film = np.zeros((100, 100, 160))

# bottom = 10
# film[:, :, 0:bottom] = 10 # bottom

# height = 40

# # film[:, :17, 0:height] = 10
# # film[:, 83:, 0:height] = 10
# # film[:17, :, 0:height] = 10
# # film[83:, :, 0:height] = 10

# slit = np.linspace(30, 0, 4)
# for i in range(4):
#     # film[:, 17+i, bottom:int(slit[i]) + bottom] = 10
#     # film[:, 79+i, bottom:int(slit[5-i]) + bottom] = 10
#     # film[17+i, :, bottom:int(slit[i]) + bottom] = 10
#     # film[79+i, :, bottom:int(slit[5-i]) + bottom] = 10
#     for j in range(10):
#         film[:, 17+i, int(slit[i])+j+ bottom] = 1/11*(10-j)
#         film[:, 79+i, int(slit[3-i])+j+ bottom] = 1/11*(10-j)
#         film[17+i, :, int(slit[i])+j+ bottom] = 1/11*(10-j)
#         film[79+i, :, int(slit[3-i])+j+ bottom] = 1/11*(10-j)
# for k in range(4):
#     film[:, 17+k, bottom:int(slit[k]) + bottom] = 10
#     film[:, 79+k, bottom:int(slit[3-k]) + bottom] = 10
#     film[17+k, :, bottom:int(slit[k]) + bottom] = 10
#     film[79+k, :, bottom:int(slit[3-k]) + bottom] = 10

# film[:, :17, 0:height] = 10
# film[:, 83:, 0:height] = 10
# film[:17, :, 0:height] = 10
# film[83:, :, 0:height] = 10
# film[:, :, height:] = 0

In [16]:
sumFilm = np.sum(film, axis=-1)

print(sumFilm.shape)

(50, 50, 100)


In [4]:
# film = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(film[:, :, :, 5]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:59843/index.html?ui=P_0x15c4182e990_0&reconnect=auto" class="pyvis…

In [17]:
etchfilm = film

In [18]:
import Multi_Species_SF_etching

In [24]:
logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_SF_etching.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                                         depoThick=90, center_with_direction=np.array([[35,100,75]]), 
                                         range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                                        reaction_type=False, param = [1.6, -0.7], N = 300000, 
                                        sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                                        celllength=1e-5, kdtreeN=5, tstep=1e-5,
                                        substrateTop=40,posGeneratorType='top', logname=logname)


# etching = testEtch.depo_position_increase_cosVel_normal(125, int(7e6), 2e-3, 5)


In [20]:
T = 300
Cm = (2*1.380649e-23*T/(27*1.66e-27) )**0.5 # (2kT/m)**0.5 27 for the Al

def max_velocity_u( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.cos(2*np.pi*random2))

def max_velocity_w( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.sin(2*np.pi*random2))

def max_velocity_v( random3):
    return -Cm*np.sqrt(-np.log(random3))

In [25]:
N = int(6e7)
velosity_matrix = np.zeros((N, 3))
tstep=1e-5
celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
# velosity_matrix[:, 1] = -1 * celllength /tstep
Random1 = np.random.rand(N)
Random2 = np.random.rand(N)
Random3 = np.random.rand(N)
velosity_matrix = np.array([max_velocity_u(Random1, Random2), \
                            max_velocity_w(Random1, Random2), \
                                max_velocity_v(Random3)]).T

energy = np.linalg.norm(velosity_matrix, axis=1)
velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)

# velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)
# FO_ratio = int(N/4)
# typeID[-FO_ratio:] = 1

# ion_ration = int(N/8)
# typeID[-ion_ration:] = 2
# velosity_matrix[-ion_ration:, 0] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 1] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

[[ 0.58315693  0.76924124 -0.26114347  0.        ]
 [-0.44839342 -0.37091701 -0.81324284  0.        ]
 [ 0.24101222  0.16195151 -0.95691422  0.        ]
 [ 0.04810873 -0.79359169 -0.60654578  0.        ]
 [-0.23433695 -0.48536024 -0.84232513  0.        ]
 [-0.96432526  0.09312919 -0.24779781  0.        ]
 [-0.32714809 -0.69598802 -0.63919856  0.        ]
 [-0.41931445  0.26144006 -0.86938167  0.        ]
 [ 0.75285343 -0.54981696 -0.36181903  0.        ]
 [ 0.69720877  0.39571669 -0.59775265  0.        ]]


In [26]:
etching = testEtch.inputParticle(125,vel_type_shuffle[:,:3], vel_type_shuffle[:,-1], 5e-3, 5)

running: 100%|██████████████████████████████████████████████████████| 100/100 [09:27<00:00, 5.67s/B]


In [28]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(etching[0][:, :, :,1]!=0, etching[0][:, :, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etching[0][:, :, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etching[0][:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:60402/index.html?ui=P_0x16b0b3740e0_4&reconnect=auto" class="pyvis…

In [13]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(etching[0][:, :25, :,1]!=0, etching[0][:, :25, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etching[0][:, :25, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etching[0][:, :25, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:61008/index.html?ui=P_0x1a0bd02bbd0_3&reconnect=auto" class="pyvis…

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)
# np.logical_and(etching[0][:, :, :, 5]>0, sumFilm[:, :, :,]>0)
depo1 = torch.Tensor(np.logical_and(etching[0][:, :, :, 5]>0, sumFilm[:, :, :,]>0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :]==5).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:52789/index.html?ui=P_0x206cdc0a4d0_5&reconnect=auto" class="pyvis…

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(etching[0][:, :, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :]==5).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:52789/index.html?ui=P_0x206cf19a4d0_2&reconnect=auto" class="pyvis…

In [ ]:
pointcloud = testEtch.get_pointcloud(sumFilm)

In [ ]:
point_cloud = pv.PolyData(pointcloud[:, 3:])
vectors = pointcloud[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=5,
)

# Display the arrowscyan

# sphere = pv.Sphere(radius=4, center=(21, 75, 114.5))
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

Widget(value='<iframe src="http://localhost:51715/index.html?ui=P_0x1f8f498a750_5&reconnect=auto" class="pyvis…

In [ ]:
depo1 = torch.Tensor(pointcloud[:,3:]).to_sparse()
depo1 = depo1.indices().numpy().T


depomesh = pv.PolyData(pointcloud[:,3:])
depomesh["radius"] = np.ones(pointcloud[:,3:].shape[0])*0.5
geom = pv.Box()


# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:51715/index.html?ui=P_0x1f8f9cd6890_6&reconnect=auto" class="pyvis…

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, 50:, :,]!=10, sumFilm[:, 50:, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, 50:, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:50932/index.html?ui=P_0x19c0f17aa10_3&reconnect=auto" class="pyvis…

In [ ]:
sumFilm = np.sum(testEtch.film, axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

ValueError: Empty meshes cannot be plotted. Input mesh has zero points. To allow plotting empty meshes, set `pv.global_theme.allow_empty_mesh = True`

In [ ]:
testEtch.parcel.shape

(7000000, 10)

In [ ]:
testEtch.parcel[106]

array([161.24791799,  84.77586652, -82.53676749,   0.80623615,
         0.42387643,  -0.4126888 ,  68.        ,  58.        ,
        99.        ,   1.        ])

In [ ]:
testEtch.parcel[100, :3] += testEtch.parcel[100, 3:6] 
print(testEtch.parcel[100])

[ 1.29022179e+02  1.60886601e+01 -1.53281836e+02  6.41898233e-01
  8.00382431e-02 -7.62601166e-01  6.30000000e+01  9.70000000e+01
  9.90000000e+01  1.00000000e+00]


In [ ]:
testEtch.parcel[10000:11000][:,8].astype(int)


array([97, 98, 95, 98, 97, 98, 99, 99, 97, 99, 97, 99, 97, 95, 96, 97, 99,
       96, 97, 99, 95, 95, 99, 98, 95, 98, 96, 96, 97, 99, 97, 97, 97, 95,
       97, 97, 98, 96, 97, 98, 97, 96, 99, 97, 97, 95, 96, 99, 96, 98, 96,
       95, 99, 99, 96, 95, 97, 97, 96, 95, 95, 98, 95, 98, 95, 96, 98, 96,
       99, 98, 98, 97, 96, 99, 97, 96, 97, 96, 97, 98, 95, 99, 95, 99, 97,
       97, 98, 95, 99, 95, 97, 99, 97, 97, 96, 96, 95, 96, 95, 99, 96, 98,
       98, 95, 95, 99, 98, 97, 97, 96, 99, 98, 99, 97, 99, 95, 97, 98, 95,
       99, 95, 98, 96, 95, 98, 96, 96, 97, 97, 96, 97, 98, 97, 96, 99, 97,
       96, 97, 99, 99, 95, 96, 97, 98, 99, 95, 95, 97, 95, 98, 95, 99, 95,
       96, 99, 95, 98, 95, 97, 96, 95, 96, 97, 97, 99, 97, 98, 98, 99, 98,
       95, 97, 98, 98, 95, 97, 95, 97, 97, 99, 95, 99, 97, 98, 95, 96, 98,
       99, 96, 99, 98, 96, 99, 95, 95, 98, 98, 99, 99, 95, 95, 98, 99, 97,
       99, 99, 95, 98, 98, 98, 99, 97, 96, 97, 99, 96, 99, 95, 97, 96, 97,
       99, 96, 97, 96, 96